<p align="left">
<img width="50%" src="https://drive.google.com/uc?export=view&id=10y3NKbbk7yt7cZDMszMt04g6NquTEa4p" alt="Carbon Logo" />
</p>


# Carbon Simulator - Whitepaper Examples

Those example relates to our [Whitepaper][wp].

[wp]:https://carbondefi.xyz/whitepaper

In [59]:
from carbon import CarbonSimulatorUI, CarbonPair as Pair
from math import sqrt

In [60]:
Sim = CarbonSimulatorUI(pair=Pair(tknb="RSK", tknq="CSH"))

We create the strategy selling RSK between 150-200 and buying it between 100-50, seeding it with 1000 CSH

In [61]:
Sim.add_strategy(
        tkn="RSK", 
        amt_sell=0, psell_start=150, psell_end=200, 
        amt_buy=1000, pbuy_start=100, pbuy_end=50
)["orders"]
print(sqrt(150*200), sqrt(50*100))

173.20508075688772 70.71067811865476


The market goes down, so the AMM sells its CSH and buys RSK

In [62]:
Sim.amm_sells("CSH", 1000)["trades"].query("aggr==True")

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,0,0,A,AMM sells 1000CSH buys 14RSK,True,True,None,1000.0,CSH,14.142136,RSK,RSKCSH,[1],1,70.710678,CSH per RSK


The market goes up again, so the AMM sells all its RSK for CSH

In [63]:
Sim.amm_sells("RSK", 14.142136)["trades"].query("aggr==True")

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,1,1,A,AMM sells 14RSK buys 2449CSH,True,True,None,14.142136,RSK,2449.489808,CSH,RSKCSH,[0],1,173.205081,CSH per RSK


In [64]:
Sim.state()["orders"][["tkn","y"]].to_dict(orient="records")

[{'tkn': 'RSK', 'y': 3.321223258972168e-27}, {'tkn': 'CSH', 'y': 2449.489808}]

The market goes down again, but only to 89.04, enough to buy 5 RSK

In [65]:
Sim.amm_buys("RSK", 5)["trades"].query("aggr==True")

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,2,2,A,AMM sells 472CSH buys 5RSK,True,True,None,471.793105,CSH,5.0,RSK,RSKCSH,[1],1,94.358621,CSH per RSK


We see that the buy-RSK curve is now stuck at the aftermentioned 89.04, and the sell-RSK curve is only filled with 5 RSK out of a capacity of 14.1, so it is stuck at 179.79, closer to its upper end at 200

In [66]:
Sim.state()["orders"]

,id,pair,tkn,y_int,y,y_unit,p_start,p_end,p_marg,p_unit,lid
0,0,RSKCSH,RSK,14.142136,5.000000,RSK,150.0,200.0,179.794460,CSH per RSK,1
1,1,RSKCSH,CSH,2449.489808,1977.696703,CSH,100.0,50.0,89.035494,CSH per RSK,0


We confirm that 179.79 is the marginal selling price

In [68]:
Sim.amm_sells("RSK", 0.000001)["trades"].query("aggr==True")

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,3,3,A,AMM sells 0RSK buys 0CSH,True,True,None,0.000001,RSK,0.00018,CSH,RSKCSH,[0],1,179.794462,CSH per RSK
